# Continuous Recirculation Direct Contact Membrane Distillation Flowsheet

<img src="MDflowsheet.png" alt="Continuous recirculation MD flowsheet" title="Continuous recirculation MD flowsheet" width="300"/>

*Figure: Continuous recirculation MD flowsheet. Adapted from: Shamlou et al.,* <em>Desalination</em>, *2022.*

Membrane Distillation (MD) is an emerging technology capable of treating highly saline wastewater, particularly relevant for handling hypersaline produced water. MD leverages a temperature gradient across a hydrophobic membrane to induce vapor pressure differences, facilitating the separation of clean water from a saline solution. The process operates at lower temperatures compared to traditional thermal desalination methods, making it well-suited for utilization of waste heat sources.

This flowsheet simulates a single Direct Contact Membrane Distillation (DCMD) model operating in continuous recirculation mode to achieve high recoveries. This mode helps overcome the low single-pass recovery limitation of MD technologies, which is typically a maximum of 10%.

The simulation includes an integrated external heat exchanger for heat recovery, optimizing energy use. The flowsheet encompasses:

* An MD module for the distillation process.
* A heater to preheat the recycle stream and makeup feed.
* A chiller to cool the feed to the cold side of the membrane, ensuring efficient separation driving force and condensation.
* An external heat exchanger to recover heat from the heated pure water in the cold loop and preheat the feed to the hot channel before it enters the heater.
* Pumps, separators, and mixers are included to support the process flow and integration.

To simulate the performance of treating produced water using Membrane Distillation (MD), this model approximates the properties of produced water with those of an NaCl solution. This approximation is for simplification and to leverage existing property packages, providing a basis for understanding the behavior and treatment efficacy of highly saline wastewater in MD systems.

To run this Jupyter Notebook and utilize the associated flowsheet for the Membrane Distillation process simulation and optimization, it's necessary to have the WaterTAP package installed on your system. Detailed installation instructions can be found in the [WaterTAP documentation](https://watertap.readthedocs.io/en/latest/getting_started.html).

To begin simulating our Membrane Distillation (MD) flowsheet, we first need to import the necessary libraries that enable the creation, manipulation, and analysis of our model.

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from pareto.models_extra.desalination_models import (
    MD_single_stage_continuous_recirculation as md_flow,
)
import ipywidgets as widgets
from IPython.display import display

Please execute the cell below to adjust the feed characteristics for the membrane distillation plant simulation. The interactive sliders allow you to tailor the salinity (kg/kg) and flowrate (kg/s) according to your specific requirements. 

In [ ]:
salinity_widget = widgets.FloatSlider(
    value=0.035, min=0.01, max=0.15, step=0.005, description="Salinity:"
)
flowrate_widget = widgets.FloatSlider(
    value=1.0, min=0.1, max=10.0, step=0.1, description="Flowrate:"
)
display(salinity_widget, flowrate_widget)

Proceed to the next cell to configure the desired recovery rate for the membrane distillation process. The slider allows you to set the recovery target within practical limits, ensuring the concentration in the solution does not exceed the solubility limit of the solute. This adjustment is dynamically calculated based on your input for feed salinity from the previous step, safeguarding against supersaturation conditions.


In [ ]:
recovery_widget = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=(1 - salinity_widget.value / 0.3),
    step=0.1,
    description="water recovery:",
)
display(recovery_widget)

Run the following cell to construct the simulation flowsheet utilizing your previously defined inputs for salinity, flowrate, and recovery. 

In [ ]:
# Function to run the flowsheet with user inputs
def run_simulation(salinity, flowrate):
    model = md_flow.build()
    md_flow.set_operating_conditions(
        model,
        feed_flow_mass=flowrate,
        feed_mass_frac_TDS=salinity,
        overall_recovery=recovery_widget.value,
    )
    md_flow.initialize_system(model)
    md_flow.optimize_set_up(model)
    md_flow.solve(model)
    return model

Execute the upcoming cell to choose the specific results you wish to review following the simulation and optimization process.

In [ ]:
# Define checkboxes for selecting the results to display
system_checkbox = widgets.Checkbox(value=False, description="Display System Results")
design_checkbox = widgets.Checkbox(value=False, description="Display Design Results")
detailed_cost_checkbox = widgets.Checkbox(
    value=False, description="Display detailed cost Results"
)
state_checkbox = widgets.Checkbox(value=False, description="Display State Results")

display(system_checkbox, detailed_cost_checkbox, design_checkbox, state_checkbox)

Execute the following cell and then click on the "Run Simulation" button to initiate the simulation and optimization of the MD flowsheet. This action triggers the comprehensive analysis of the flowsheet based on your specified inputs.

In [ ]:
# Configuration for the "Run Simulation" button and its output handling
run_button = widgets.Button(description="Run Simulation")
output = widgets.Output()


def on_run_button_clicked(b):
    with output:
        output.clear_output()
        model = run_simulation(salinity_widget.value, flowrate_widget.value)
        # Display selected results based on checkboxes
        if system_checkbox.value:
            md_flow.display_system(model)
        if design_checkbox.value:
            md_flow.display_design(model)
        if state_checkbox.value:
            md_flow.display_state(model)
        if detailed_cost_checkbox.value:
            md_flow.display_detailed_costs(model)


run_button.on_click(on_run_button_clicked)
display(run_button, output)